# Tableau Project (Criterio 4: df_capacity)

In [1]:
#Importo librerias.
import pymongo
import pandas as pd
import re

#Creo cursor y dataframe inicial.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
cursor = db.companies.find()
data = list(cursor)
df = pd.DataFrame(data)

In [2]:
#Análisis de atributos
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

In [3]:
#Primera fase de limpieza de datos.
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)

#Eliminación de registros correspondientes a empresas en 'deadpool'.
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]

#Eliminación de registros con valor 'null'
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
#Verificación de la longitud del nuevo dataset.
print(len(df_nulls))

7934


In [4]:
#Elección de columnas relevantes para el criterio.
col_cat = ['games_video']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)

#Verificación de la longitud del nuevo dataset.
print(len(df_cat))

525


In [5]:
#Definición de funciones para modificación de tipo de datos.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer

In [6]:
#Aplicación de funciones sobre los registros.
df_regout = df_cat.copy()
df_regout['total_money_raised'] = df_regout['total_money_raised'].apply(str_to_int)
df_regout['founded_year'] = df_regout['founded_year'].apply(float_to_int)

#Eliminación de registros irrelevantes según criterio.
#df_regout = df_regout[df_regout.total_money_raised == 0]
#df_regout = df_regout[df_regout.number_of_employees > 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]

#Verificación de la longitud del nuevo dataset.
print(len(df_regout))

406


In [7]:
#Función para la creación de los valores de ubicación en formato GeoJson.
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0

In [8]:
#Aplicación de función para la creación de los valores de ubicación en formato GeoJson.
df_colout = df_regout.copy()
df_colout['offices'] = df_colout['offices'].apply(nested_to_list)

In [9]:
#Eliminación de valores de GeoJson nulos.
df_geoout = df_colout.copy()
df_geoout = df_geoout[df_geoout.offices != 0]

#Verificación de la longitud del nuevo dataset.
print(len(df_geoout))

#Vista del dataset definitivo antes de generar .json
display(df_geoout.head())

309


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
16,games_video,2006,Joost,0.0,"{'type': 'Point', 'coordinates': [-74.0094471,...",45000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000
51,games_video,2005,YouTube,0.0,"{'type': 'Point', 'coordinates': [-122.426804,...",11000
52,games_video,2007,Livestream,120.0,"{'type': 'Point', 'coordinates': [-73.995625, ...",14000
54,games_video,2007,AdaptiveBlue,15.0,"{'type': 'Point', 'coordinates': [-74.3372, 40...",24000


In [10]:
#Creación del fichero .json para generar nueva 'collection' en MongoDB
df_geoout.to_json('capacity.json', orient="records", lines=True)

### Se importa .json en MongoDB para generar collection y aplicar operador geoespacial '$near'
Nombre de la collection creada: **capacity**

In [11]:
#Importación de datos de centros formativos top en creación de contenidos
df_school = pd.read_csv('education.csv', sep=',', index_col=False)

#Vista del dataset
display(df_school)

,Ranking,University,Latitude,Longitude
0,1,Abilene Christian University,32.469736,-99.710292
1,2,Laguna College of Art and Design,33.561689,-117.774531
2,3,Oklahoma Christian University,35.612447,-97.471548
3,4,Southwestern College,37.249499,-96.978204
4,5,Champlain College,44.473120,-73.206329
5,6,Walla Walla University,46.046460,-118.393585
6,7,The Savannah College of Art and Design,32.069267,-81.095983
7,8,Ferris State University,43.683414,-85.486372
8,9,Southern Arkansas University,33.293291,-93.234082
9,10,Columbia College Chicago,41.874237,-87.627028


In [27]:
#Se define función para aplicar operador geoespacial '$near' para comparar contra los centros
def concentration(school):
    cursor_near = db.capacity.find({
      "offices": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [df_school['Longitude'][school],df_school['Latitude'][school]]
          },
          "$minDistance": 0,
          "$maxDistance":5000
        }
      }
    })
    return list(cursor_near)

In [28]:
#Aplicación de la función para cada documento de la colección respecto al resto de documentos
#cursor_geojson = db.capacity.find()
clusters = []
for element in range(len(df_school['Longitude'])):
    clusters.append(len(concentration(element)))

#Verificación de la longitud del cursor (total de iteraciones).
print(len(clusters),'\n')

#Visualización del total de coincidencias para cada documento.
print(clusters,'\n')

#Máximo número de coincidencias.
print(max(clusters),'\n')

10 

[0, 1, 0, 0, 0, 0, 0, 0, 0, 3] 

3 



In [29]:
#Extracción de coordenadas geográficas con mayor número de coincidencias dentro del criterio.
target = list(db.capacity.find())
targets = []
for i in range(len(clusters)):
    if clusters[i] == max(clusters):
        targets.append([target[i]['offices']['coordinates'][0], target[i]['offices']['coordinates'][1]])

#Visualización de los pares de coordenadas resultantes.
print(targets)

[[-122.323408, 47.615313]]


In [30]:
#Creación de dataframe para Plateau.
long_lat = ['Longitude','Latitude']
df_capacity = pd.DataFrame(targets, columns=long_lat)
df_capacity['Criteria'] = 'Capacity'

#Visualización del dataframe final dentro del criterio.
display(df_capacity)

,Longitude,Latitude,Criteria
0,-122.323408,47.615313,Capacity


In [31]:
#Creación de fichero .json para visualización en Plateau.
df_capacity.to_json('df_capacity.json', orient="records", lines=True)